# Question Answering

## Overview
Recall the overall workflow for retrieval augmented generation (RAG):

We discussed Document Loading and Splitting as well as Storage and Retrieval.

Let's load our vectorDB.

In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['OPENAI_API_KEY']

The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023. LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [2]:
# import datetime
# current_date = datetime.datetime.now().date()
# if current_date < datetime.date(2023, 9, 2):
#     llm_name = "gpt-3.5-turbo-0301"
# else:
#     llm_name = "gpt-3.5-turbo"
# print(llm_name)

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding_endpoint = os.getenv("EMBEDDING_AZURE_OPENAI_ENDPOINT")
embedding_api_key = os.getenv("EMBEDDING_API_KEY")
# Initialize OpenAI Embeddings
embedding = AzureOpenAIEmbeddings(
    deployment="ada", # Make sure to use your embedding model deployment name
    azure_endpoint=embedding_endpoint,
    openai_api_key=embedding_api_key,
    chunk_size=16, # 16 is the max parallel embeddings Azure can handle for now
)
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
print(vectordb._collection.count())

209


In [5]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [6]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo", # Make sure to use your chat model deployment name
    temperature=0,
)

## RetrievalQA chain

In [7]:
from langchain.chains import RetrievalQA

In [8]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [9]:
result = qa_chain({"query": question})

/Users/baptiste.pasquier/Library/Caches/pypoetry/virtualenvs/langchain-chat-with-your-data-Z0fYq3G3-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [10]:
result["result"]

'The major topics for this class are machine learning and its various subfields.'

## Prompt

In [11]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [12]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [13]:
question = "Is probability a class topic?"

In [14]:
result = qa_chain({"query": question})

In [15]:
result["result"]

'Yes, probability is a topic that will be covered in the class. Thanks for asking!'

In [16]:
result["source_documents"][0]

Document(page_content="of this class will not be very program ming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octa ve. I'll say a bit more about that later.  \nI also assume familiarity with basic proba bility and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what ra ndom variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort of as  a refresher course under prerequisite class. \nI'll say a bit more about that later as well.  \nLastly, I also assume familiarity with basi c linear algebra. And again, most undergraduate \nlinear algebra courses are more than enough. So if you've taken courses like Math 51, \n103, Math 113 or 

## RetrievalQA chain types

In [17]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [18]:
result = qa_chain_mr({"query": question})

In [19]:
result["result"]

'Based on the provided information, it is not clear whether probability is a specific topic covered in the class.'

If you wish to experiment on the LangChain plus platform:

- Go to [langchain plus platform](https://smith.langchain.com/) and sign up
- Create an API key from your account's settings
- Use this API key in the code below
- uncomment the code

Note, the endpoint in the video differs from the one below. Use the one below.

In [20]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "..." # replace dots with your api key

In [21]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.langchain.plus/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.langchain.plus/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.langchain.plus/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.langchain.plus/runs/batch\', \'{"detail":"Invalid API key"}\')')


'Based on the provided information, it is not clear whether probability is a specific topic covered in the class.'

In [22]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.langchain.plus/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.langchain.plus/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.langchain.plus/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.langchain.plus/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.langchain.plus/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.langchain.plus/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.langchain.plus/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.langchain.plus/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs:

'Based on the new context provided, it is likely that probability is indeed a class topic. The instructor mentions going over statistics and algebra as refreshers in the discussion sections, which suggests that these topics are relevant to the main lectures. Additionally, the instructor mentions teaching extensions for the material covered in the main lectures, indicating that probability may be one of those extensions.'

## RetrievalQA limitations
QA fails to preserve conversational history.

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [24]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a topic that will be covered in this class. The instructor assumes familiarity with basic probability and statistics, so it is expected that students have prior knowledge of random variables, expectation, variance, and other related concepts.'

In [25]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites are needed because they provide the foundational knowledge and skills necessary to understand and apply the concepts and techniques taught in the class. \n\nBasic knowledge of computer science and computer skills is important because machine learning algorithms often involve programming and working with data. Understanding big-O notation is essential for analyzing the efficiency and scalability of algorithms.\n\nFamiliarity with probability and statistics is necessary because machine learning involves making predictions and analyzing data, which requires understanding concepts such as random variables, expectation, and variance.\n\nBasic knowledge of linear algebra is important because many machine learning algorithms involve manipulating matrices and vectors. Understanding concepts such as matrix multiplication, matrix inverse, and eigenvectors is crucial for working with these algorithms.\n\nOverall, these prerequisites ensure that students have the necessary backg

Note, The LLM response varies. Some responses do include a reference to probability which might be gleaned from referenced documents. The point is simply that the model does not have access to past questions or answers, this will be covered in the next section.